In [1]:
import logging
import argparse
import math
import os
import torch
import random
import numpy as np
import json, jsonlines
import pickle
import time
import copy
import transformers
from transformers import BertTokenizer
from knowledge_neuron.knowledge_neurons.src.custom_bert import BertForMaskedLM
import torch.nn.functional as F
from knowledge_neuron.knowledge_neurons.src import *

from IPython.display import clear_output
import matplotlib.pyplot as plt
from datasets import load_dataset

from main import *

class Params:
    def __init__(this):
        this.temp = None

* in below cell change the relation type and edit method as required

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

args = Params()
args.seed = 42
args.output_dir = './results'
args.output_prefix = 'TREx-all'
args.bert_model = 'bert-base-cased'
args.do_lower_case = False
args.tmp_data_path = './knowledge_neuron/knowledge_neurons/data/PARAREL/data_all_allbags.json'
args.data_path = './knowledge_neuron/knowledge_neurons/data/PARAREL/data_all.json'
args.debug = 100000
args.pt_relation = 'P101'
args.max_seq_length = 128
args.get_pred = True
args.batch_size = 20
args.num_batch = 1
args.get_ig_pred = True
args.get_ig_gold = True
args.get_base = True
args.phaseone_processed_data_path = './data'
args.relative_attribution_threshold = 0.2
#args.relations = ['P1376'] #'P463',
args.kns_results_folder = './kns_results'
args.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
args.n_gpu = 1
args.model_save_path = './temp_model_save'
args.relation_data_path = './knowledge_neuron/knowledge_neurons/data/LAMA/raw_data/TREx'

# prepare eval set
if os.path.exists(args.tmp_data_path):
    with open(args.tmp_data_path, 'r') as f:
        eval_bag_list_perrel = json.load(f)
else:
    with open(args.data_path, 'r') as f:
        eval_bag_list_all = json.load(f)
    # split bag list into relations
    eval_bag_list_perrel = {}
    for bag_idx, eval_bag in enumerate(eval_bag_list_all):
        bag_rel = eval_bag[0][2].split('(')[0]
        if bag_rel not in eval_bag_list_perrel:
            eval_bag_list_perrel[bag_rel] = []
        if len(eval_bag_list_perrel[bag_rel]) >= args.debug:
            continue
        eval_bag_list_perrel[bag_rel].append(eval_bag)
    with open(args.tmp_data_path, 'w') as fw:
        json.dump(eval_bag_list_perrel, fw, indent=2)



tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
model = BertForMaskedLM.from_pretrained('bert-base-cased')
model = model.to(device)
print(model)

args.relations = ['P1376'] # change the relation types which you wish to edit
args.edit_type = None

model = main(model,tokenizer,eval_bag_list_perrel,args,edit_type=args.edit_type,display=True)


In [ ]:
get_kns_statistics(args.kns_results_folder,True)